# 0 Installing dependencies

In [ ]:
!pip install Pillow
!pip install pandas
!pip install opencv-python
# make sure you have correct cuda and cudnn versions installed. refer to https://pytorch.org/get-started/locally/ and NVIDIA installation of cuda and cudnn
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
!pip install tqdm
!pip install matplotlib
!pip install torch-summary
!git clone https://github.com/EscVM/OIDv4_ToolKit.git
!pip install -r OIDv4_ToolKit/requirements.txt
!pip install onnx-simplifier

# 1 Setup

## 1.1 Choose a model name

In [ ]:
model_name = 'Yolo_FastestV2_V3_custom_phone_only'    # note: do not use dash (minus sign) in the name of the model

## 1.2 Choose object classes

In [ ]:
# Enter objects you would like your model to detect. 600 are available and visible at OID/csv_folder/class-descriptions-boxable.csv 
classes = ['Camera', 'Mobile phone']
# set to true if mobile phone and camera should be considered as one camera class. else false
cam_and_phone_combined = True
# set to true to use custom labelled mobile phone and camera set. make sure that it is in "custom_dataset" folder and in the same directory as this notebook
custom_dataset = True
# Enter number of images you want (if you want) to download for each object. Keep them in the same order. else simply ignore
limit_images_to = [5000, 5000]

## 1.3 Imports

In [ ]:
import fnmatch
import math
import shutil
import os 
import PIL
import pandas as pd
import cv2
import time
import torch
import sys
import random
import string
import glob
import pickle
import xml.etree.ElementTree as ET
from random import randint
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
from PIL import Image
from pathlib import Path
sys.path.insert(0, './Yolo-FastestV2-main')
import model.detector as det
import utils.utils 

## 1.4 Paths

In [ ]:
paths = {
'DATASET_DIR': os.path.join('OID', 'Dataset', 'train'),
'YOLO_MAIN_DIR': os.path.join('Yolo-FastestV2-main'),
'TRAIN_IMAGES_DEST': os.path.join('Yolo-FastestV2-main', 'train'),
'VAL_IMAGES_DEST': os.path.join('Yolo-FastestV2-main', 'val'),
'YOLO_DATA_DIR': os.path.join('Yolo-FastestV2-main', 'data')
}

for aClass in classes:
    paths['DATASET_' + aClass.upper().replace(' ', '_')] = os.path.join(paths['DATASET_DIR'], aClass)
    paths['ALL_IMGS_' + aClass.upper().replace(' ', '_')] = os.path.join(paths['DATASET_DIR'], aClass, aClass.replace(' ', '_') +'_images')
    paths['ALL_UPDATED_LABELS_' + aClass.upper().replace(' ', '_')] = os.path.join(paths['DATASET_DIR'], aClass, 'Label_updated')
    paths['LOCAL_TRAIN_IMGS_' + aClass.upper().replace(' ', '_')] = os.path.join(paths['DATASET_DIR'], aClass, 'train_imgs_local')
    paths['LOCAL_TRAIN_LABELS_' + aClass.upper().replace(' ', '_')] = os.path.join(paths['DATASET_DIR'], aClass, 'train_labels_local')
    paths['LOCAL_VAL_IMGS_' + aClass.upper().replace(' ', '_')] = os.path.join(paths['DATASET_DIR'], aClass, 'val_imgs_local')
    paths['LOCAL_VAL_LABELS_' + aClass.upper().replace(' ', '_')] = os.path.join(paths['DATASET_DIR'], aClass, 'val_labels_local')
    paths['YOLO_LABELS_' + aClass.upper().replace(' ', '_')] = os.path.join(paths['DATASET_DIR'], aClass, 'Yolo_format_labels')

# 2 Labeled dataset creation

## 2.1 Downloading the dataset

In [ ]:
print('Copy command(s) below and run in the terminal. Make sure to be in the directory where this notebook is.\n')

for classNum in range(len(classes)):
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    command = ('python OIDv4_ToolKit/main.py downloader --classes ' + '\"' + classes[classNum] + '\"' + ' --type_csv train --limit ' + str(limit_images_to[classNum]))
    print(command)
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

## 2.2 Creating label files with multiple objects in an image

In [ ]:
# Parts of this section were mode by modifying https://github.com/EscVM/OIDv4_ToolKit

In [ ]:
# read csv files containing labels and classes
df_val = pd.read_csv(os.path.join('OID', 'csv_folder', 'train-annotations-bbox.csv'))
df_classes = pd.read_csv(os.path.join('OID', 'csv_folder', 'class-descriptions-boxable.csv'), header=None)

In [ ]:
class_code = [0] * len(classes)
for classNum in range(len(classes)):
    class_code[classNum] = df_classes.loc[df_classes[1] == classes[classNum]].values[0][0]

In [ ]:
# create a dictionary of images needed for labelling
groups_dict = {}
keys = ['0'] * len(classes)
for i in range(len(classes)):
    keys[i] = 'groups_' + classes[i]

values = class_code
for i in range(len(keys)):
        groups_dict[keys[i]] = df_val[(df_val.LabelName == values[i])].groupby(df_val.ImageID)

In [ ]:
# put images in one folder for a clean directory structure
for aClass in classes:
    # remove label folder since updated version will be made
    if os.path.exists(os.path.join(paths['DATASET_' + aClass.upper().replace(' ', '_')], 'Label')):
        shutil.rmtree(os.path.join(paths['DATASET_' + aClass.upper().replace(' ', '_')], 'Label'))
    !cd {os.path.join(paths['DATASET_' + aClass.upper().replace(' ', '_')])} && mkdir {aClass.replace(' ', '_')  + '_images'}
    image_list = [f.split('.')[0] for f in os.listdir(os.path.join(paths['DATASET_DIR'], aClass)) if f.endswith('.jpg')]
    for image in image_list:
        os.rename(os.path.join(paths['DATASET_' + aClass.upper().replace(' ', '_')], image + '.jpg'), os.path.join(paths['ALL_IMGS_' + aClass.upper().replace(' ', '_')], image + '.jpg'))

In [ ]:
# create labels for downloaded classes. by default when label files are made by OIDv4_ToolKit it only has one class in them.
# this function takes care of that and can make multiple labels of different classes in an image of interest.
def labelUpdater(): 
    groups = list(groups_dict.values())
    for i in range(len(groups)):
        # copies are made to not affect original lists when reshuffling entries in them
        copy_groups = list.copy(groups)
        copy_groups.insert(0, copy_groups.pop(copy_groups.index(copy_groups[i])))
        copy_classes = list.copy(classes)
        copy_classes.insert(0, copy_classes.pop(copy_classes.index(copy_classes[i])))

        downloaded_images_list = [f.split('.')[0] for f in os.listdir(os.path.join(paths['ALL_IMGS_' + copy_classes[0].upper().replace(' ', '_')])) if f.endswith('.jpg')]
        images_label_list = list(set(downloaded_images_list))

        for image in tqdm(downloaded_images_list):
            try:
                current_image_path = os.path.join(paths['ALL_IMGS_' + copy_classes[0].upper().replace(' ', '_')], image + '.jpg')
                dataset_image = cv2.imread(current_image_path)
                # get boxes for current image
                boxes = copy_groups[0].get_group(image.split('.')[0])[['XMin', 'XMax', 'YMin', 'YMax']].values.tolist()
                file_name = str(image.split('.')[0]) + '.txt'
                file_path = os.path.join(paths['ALL_UPDATED_LABELS_' + copy_classes[0].upper().replace(' ', '_')], file_name)
                if os.path.isfile(file_path):
                    f = open(file_path, 'a')
                else:
                    f = open(file_path, 'w')

                for box in boxes:
                    box[0] *= int(dataset_image.shape[1])
                    box[1] *= int(dataset_image.shape[1])
                    box[2] *= int(dataset_image.shape[0])
                    box[3] *= int(dataset_image.shape[0])

                    # each row in a file is name of the class_name, XMin, YMix, XMax, YMax (left top right bottom)
                    print(copy_classes[0].replace(' ', '_'), box[0], box[2], box[1], box[3], file=f)

                # add boxes from other classes to the label file
                for categoryNum in range(len(copy_classes) - 1):
                    try:
                        boxes = copy_groups[categoryNum + 1].get_group(image.split('.')[0])[['XMin', 'XMax', 'YMin', 'YMax']].values.tolist()

                        for box in boxes:
                            box[0] *= int(dataset_image.shape[1])
                            box[1] *= int(dataset_image.shape[1])
                            box[2] *= int(dataset_image.shape[0])
                            box[3] *= int(dataset_image.shape[0])

                        # each row in a file is name of the class_name, XMin, YMix, XMax, YMax (left top right bottom)
                        print(copy_classes[categoryNum + 1].replace(' ', '_'), box[0], box[2], box[1], box[3], file=f)
                    except Exception as e:
                        pass    

            except Exception as e:
                pass

In [ ]:
# searching labels in a file with hundreds of millions of entries, will take some time
print('This might take a long time depending on the size of the dataset. Please wait...')
if custom_dataset:
    shutil.rmtree(os.path.join('OID', 'Dataset', 'train', 'Camera'))
    !cd {os.path.join('OID', 'Dataset', 'train')} && mkdir Camera
    shutil.copytree(os.path.join('custom_dataset', 'Camera', 'Camera_images'), os.path.join('OID', 'Dataset', 'train', 'Camera', 'Camera_images'))

for aClass in classes:
    if os.path.exists(os.path.join(paths['ALL_UPDATED_LABELS_' + aClass.upper().replace(' ', '_')])):
        shutil.rmtree(os.path.join(paths['ALL_UPDATED_LABELS_' + aClass.upper().replace(' ', '_')]))
    !cd {os.path.join(paths['DATASET_' + aClass.upper().replace(' ', '_')])} && mkdir Label_updated
labelUpdater()

if custom_dataset:
    shutil.rmtree(os.path.join('OID', 'Dataset', 'train', 'Mobile phone'))
    !cd {os.path.join('OID', 'Dataset', 'train')} && mkdir "Mobile phone"
    shutil.copytree(os.path.join('custom_dataset', 'Mobile phone', 'train and val', 'Mobile_phone_images'), os.path.join('OID', 'Dataset', 'train', 'Mobile phone', 'Mobile_phone_images'))
    shutil.copytree(os.path.join('custom_dataset', 'Mobile phone', 'train and val', 'Label_updated'), os.path.join('OID', 'Dataset', 'train', 'Mobile phone', 'Label_updated'))

## 2.3 Data augmentation

In [ ]:
# set to True if you would like to use data augmentation
add_data_augmentation = False
# randomly select different augmentations methods for each image from preferred augmentation methods list (aug_params)
randomize_augmentations = True
# select factor by which corresponding classes will be increased with data augmentation. order should be the same as in section 1.2
increase_class_by_factor = [3, 3]

In [ ]:
if add_data_augmentation: 
    if len(increase_class_by_factor) != len(classes):
        raise NameError('There should be the same number of factors as classes')
    if (not all((factor >= 1 and factor <= 8) for factor in increase_class_by_factor)) or (not all(isinstance(factor, int) == True for factor in increase_class_by_factor)):
        raise NameError('All factors need to be integers between 1 and 8')

In [ ]:
# set data augmentation methods you wish to use to True. Set at least 3 options to True.
if add_data_augmentation:
    mirror_flips = True
    rotations = True
    brightness_change = True
    saturation_change = True
    contrast_change = True
    noise_gauss = True
    noise_salt_and_pepper = True
    
    aug_params = [mirror_flips, rotations, brightness_change, saturation_change, contrast_change, noise_gauss, noise_salt_and_pepper]
    
    if sum(aug_params) < 3:
        aug_params = []
        raise NameError('At least 3 data augmentation parameters need to be set to true') 

In [ ]:
# randomize data augmentation for each image
def randomizeAugmentation(aug_factor):
    results = []
    
    for i in range(aug_factor - 1):
        while(True):
            aug_params_local = list.copy(aug_params)
            if randomize_augmentations:
                for paramIndex in range(len(aug_params_local)):
                    if aug_params_local[paramIndex]:
                        # flip a random setting to false with 40% chance
                        k = random.randint(0, 4)
                        if k < 2 and sum(aug_params_local) >= 2:
                            aug_params_local[paramIndex] = False
            
            if randomize_augmentations:
                # check if this augmentation was not already used on an image before
                if not results:
                    results.append(aug_params_local)
                    break
                else:
                    flag = False
                    for result in results:
                        if len(result) == sum([1 for i, j in zip(result, aug_params_local) if i == j]):
                            flag = True
                            break    # continue while
                    if not flag:
                        results.append(aug_params_local)
                        break    # break while  
            else: 
                results.append(aug_params_local)
                break
    return(results)

In [ ]:
# functions were obtained from https://towardsdatascience.com/data-augmentation-compilation-with-python-and-opencv-b76b1cd500e0
def colorjitter(img, cj_type):
    '''
    ### Different Color Jitter ###
    img: image
    cj_type: {b: brightness, s: saturation, c: constast}
    '''
    if cj_type == 'b':
        value = np.random.choice(np.array([-40, -30, 30, 40]))
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(hsv)
        if value >= 0:
            lim = 255 - value
            v[v > lim] = 255
            v[v <= lim] += value
        else:
            lim = np.absolute(value)
            v[v < lim] = 0
            v[v >= lim] -= np.absolute(value)

        final_hsv = cv2.merge((h, s, v))
        img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
        return img
    
    elif cj_type == 's':
        value = np.random.choice(np.array([-40, -30, 30, 40]))
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(hsv)
        if value >= 0:
            lim = 255 - value
            s[s > lim] = 255
            s[s <= lim] += value
        else:
            lim = np.absolute(value)
            s[s < lim] = 0
            s[s >= lim] -= np.absolute(value)

        final_hsv = cv2.merge((h, s, v))
        img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
        return img
    
    elif cj_type == 'c':
        brightness = 10
        contrast = random.randint(40, 100)
        dummy = np.int16(img)
        dummy = dummy * (contrast/127+1) - contrast + brightness
        dummy = np.clip(dummy, 0, 255)
        img = np.uint8(dummy)
        return img
    
def noisy(img, noise_type):
    '''
    ### Adding Noise ###
    img: image
    cj_type: {gauss: gaussian, sp: salt & pepper}
    '''
    if noise_type == 'gauss':
        image=img.copy() 
        mean=0
        st=0.2
        gauss = np.random.normal(mean,st,image.shape)
        gauss = gauss.astype('uint8')
        image = cv2.add(image,gauss)
        return image
    
    elif noise_type == 'sp':
        image=img.copy() 
        prob = 0.02
        if len(image.shape) == 2:
            black = 0
            white = 255            
        else:
            colorspace = image.shape[2]
            if colorspace == 3:  # RGB
                black = np.array([0, 0, 0], dtype='uint8')
                white = np.array([255, 255, 255], dtype='uint8')
            else:  # RGBA
                black = np.array([0, 0, 0, 255], dtype='uint8')
                white = np.array([255, 255, 255, 255], dtype='uint8')
        probs = np.random.random(image.shape[:2])
        image[probs < (prob / 2)] = black
        image[probs > 1 - (prob / 2)] = white
        return image

In [ ]:
# augment images and labels
def augmentFunction(augmentation, index, aClass, imageName):
    image = cv2.imread(os.path.join(paths['ALL_IMGS_' + aClass.upper().replace(' ', '_')], imageName))

    # initialize to values that cannot be taken
    flip = -2
    rotation = -2
    
    # perform augmentations set to true
    if augmentation[0]:
        flip = random.randint(-1,1)
        image = cv2.flip(image, flip)
    if augmentation[1]:
        rotation = random.randint(0,2)
        if rotation == 0:
            image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
        elif rotation == 1:
            image = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
        else:
            image = cv2.rotate(image, cv2.ROTATE_180)
    if augmentation[2]:
        image = colorjitter(image, cj_type = 'b')
    if augmentation[3]:
        image = colorjitter(image, cj_type = 's')      
    if augmentation[4]:
        image = colorjitter(image, cj_type = 'c')        
    if augmentation[5]:
        image = noisy(image, 'gauss')      
    if augmentation[6]:
        image = noisy(image, 'sp')
            
    # add image to the dataset
    cv2.imwrite(os.path.join(paths['ALL_IMGS_' + aClass.upper().replace(' ', '_')], imageName.split('.')[0] + '_aug_' + str(index) + '.jpg'), image)

    label_location = os.path.join(paths['ALL_UPDATED_LABELS_' + aClass.upper().replace(' ', '_')], imageName.split('.')[0] + '.txt')
    dest_location = os.path.join(paths['ALL_UPDATED_LABELS_' + aClass.upper().replace(' ', '_')], imageName.split('.')[0] + '_aug_' + str(index) + '.txt')
    
    # make labels for augmented images and take care of boxes that need to be moved due to rotations and flips
    if (flip == -2 and rotation == -2) or (flip == -1 and rotation == 2):
        shutil.copyfile(label_location, dest_location)
    else:
        with open(label_location, 'r') as fRead:
            with open(dest_location, 'a') as fWrite:
                imagePIL = PIL.Image.open(os.path.join(paths['ALL_IMGS_' + aClass.upper().replace(' ', '_')], imageName))
                w, h = imagePIL.size
                lines = fRead.readlines()
                for line in lines:
                    line_split = line.strip().split(' ')
                    # x1 is line_split[1]
                    # x2 is line_split[3]
                    # y1 is line_split[2]
                    # y2 is line_split[4]
                    # w-x1 is str(w-float(line_split[1]))
                    # w-x2 is str(w-float(line_split[3]))
                    # h-y1 is str(h-float(line_split[2]))
                    # h-y2 is str(h-float(line_split[4]))
                    if (flip == -2 and rotation == 0) or (flip == -1 and rotation == 1):      
                        # h-y2    x1    h-y1    x2        
                        fWrite.writelines(line_split[0] + ' ' + str(h-float(line_split[4])) + ' ' + line_split[1] + ' ' + str(h-float(line_split[2])) + ' ' + line_split[3])
                    elif (flip == -2 and rotation == 1) or (flip == -1 and rotation == 0):
                        # y1    w-x2    y2    w-x1
                        fWrite.writelines(line_split[0] + ' ' + line_split[2] + ' ' + str(w-float(line_split[3])) + ' ' + line_split[4]  + ' ' + str(w-float(line_split[1])))
                    elif (flip == -2 and rotation == 2) or (flip == -1 and rotation == -2):  
                        # w-x2    h-y2    w-x1    h-y1        
                        fWrite.writelines(line_split[0] + ' ' + str(w-float(line_split[3])) + ' ' + str(h-float(line_split[4])) + ' ' + str(w-float(line_split[1]))  + ' ' + str(h-float(line_split[2])))
                    elif (flip == 0 and rotation == -2) or (flip == 1 and rotation == 2):  
                        # x1    h-y2    x2    h-y1
                        fWrite.writelines(line_split[0] + ' ' + line_split[1] + ' ' + str(h-float(line_split[4])) + ' ' + line_split[3]  + ' ' + str(h-float(line_split[2])))
                    elif (flip == 0 and rotation == 0) or (flip == 1 and rotation == 1):  
                        # y1    x1    y2    x2   
                        fWrite.writelines(line_split[0] + ' ' + line_split[2] + ' ' + line_split[1] + ' ' + line_split[4]  + ' ' + line_split[3])
                    elif (flip == 0 and rotation == 1) or (flip == 1 and rotation == 0):         
                        # h-y2    w-x2    h-y1    w-x1         
                        fWrite.writelines(line_split[0] + ' ' + str(h-float(line_split[4])) + ' ' + str(w-float(line_split[3])) + ' ' + str(h-float(line_split[2]))  + ' ' + str(w-float(line_split[1])))
                    elif (flip == 0 and rotation == 2) or (flip == 1 and rotation == -2): 
                        # w-x2    y1    w-x1    y2 
                        fWrite.writelines(line_split[0] + ' ' + str(w-float(line_split[3])) + ' ' + line_split[2] + ' ' + str(w-float(line_split[1]))  + ' ' + line_split[4])
       
                    if lines.index(line) != (len(lines) - 1):
                        fWrite.writelines('\n')

In [ ]:
if add_data_augmentation:
    # can take hours with large dataset or large augmentation factor
    print('This will certainly take a long time and even longer if dataset is large. Please wait...')    
    for factorIndex in range(len(increase_class_by_factor)):
        if increase_class_by_factor[factorIndex] == 1:
            continue

        all_images = os.listdir(paths['ALL_IMGS_' + classes[factorIndex].upper().replace(' ', '_')])
        for image in tqdm(all_images):
            aug_param_list = randomizeAugmentation(increase_class_by_factor[factorIndex])
    
            count_aug = 0
            for augmentation in aug_param_list:
                if not randomize_augmentations:
                    augmentFunction(augmentation, count_aug, classes[factorIndex], image)
                    count_aug = count_aug + 1
                else:
                    augmentFunction(augmentation, aug_param_list.index(augmentation), classes[factorIndex], image)

## 2.5 Small object data augmentation

In [ ]:
# adds cropped and smaller in size copies of objects to the image -> better small object detection

In [ ]:
small_obj_data_augmentation = False 

# number of added small objects per image
num_of_small_obj = 1
# enter classes you already use and would like to augment using this method. if empty -> all classes are used
classes_for_small_obj_data_augmentation = []
# minumum number of pixels needed for an orginal object which will be cropped and made 1.3-2.5 times smaller (see smallObjectAugmentation function)
min_pixel_area = 4000 

In [ ]:
if small_obj_data_augmentation:
    for aClass in classes_for_small_obj_data_augmentation:
        if aClass not in classes:
            raise NameError('Class \"' + aClass + '\" given in classes_for_small_obj_data_augmentation is not used in this model (check section 1.2)') 

In [ ]:
def smallObjectAugmentation(aClass):
    img_directory = os.path.join(paths['ALL_IMGS_' + aClass.upper().replace(' ', '_')])
    images = os.listdir(img_directory)

    for image in tqdm(images):
        ori_img = cv2.imread(os.path.join(img_directory, image))
        label_location = os.path.join(paths['ALL_UPDATED_LABELS_' + aClass.upper().replace(' ', '_')], image.split('.')[0] + '.txt')

        all_boxes = []
        all_boxes_len = 0;
        with open(label_location, 'r') as fRead:
            lines = fRead.readlines()
            for line in lines:
                line_split = line.strip().split(' ')
                all_boxes.append([line_split[0], int(float(line_split[1])), int(float(line_split[2])), int(float(line_split[3])), int(float(line_split[4]))])
                all_boxes_len = len(all_boxes) 

        for i in range (0, num_of_small_obj):
            obj_too_small = False
            random_box = []
            for count in range(0, 5):
                random_box = random.choice(all_boxes[0:all_boxes_len])
                # if area is more than 4000 pixels
                if (random_box[3]-random_box[1])*(random_box[4]-random_box[2]) >= min_pixel_area:
                    break;
                elif count == 4:
                    obj_too_small = True
                    break

            if obj_too_small:
                break

            crop_img = ori_img[random_box[2]:random_box[4], random_box[1]:random_box[3]]
            # resize object by this factor
            random_resize = round(random.uniform(1.3,2.5), 2)
            crop_img = cv2.resize(crop_img, (int(crop_img.shape[1]/random_resize), int(crop_img.shape[0]/random_resize)))

            no_space_counter = 0
            no_space = False
            while (True):
                # if object was randomly placed 10 times but it always overlapped with existing objects
                if no_space_counter > 10:
                    no_space = True
                    break

                no_space_counter = no_space_counter + 1
                y = random.choice([i for i in range(0,ori_img.shape[0] - crop_img.shape[0])])
                x = random.choice([i for i in range(0,ori_img.shape[1] - crop_img.shape[1])])

                all_boxes_passed = False
                for box in all_boxes:
                    #a.x1 > b.x2 || a.x2 < b.x1 || a.y1 > b.y2 || a.y2 < b.y1, no overlaps with other objects
                    if x > box[3] or x+crop_img.shape[1] < box[1] or y > box[4] or y+crop_img.shape[0] < box[2]:
                        if all_boxes.index(box) == len(all_boxes) - 1:
                            all_boxes_passed = True
                            all_boxes.append([box[0], x, y, x+crop_img.shape[1], y+crop_img.shape[0]])

                        else:
                            continue
                    else:
                        break

                if all_boxes_passed:
                    break

            if no_space:
                break

            ori_img[y:y+crop_img.shape[0],x:x+crop_img.shape[1]] = crop_img

            cv2.imwrite(os.path.join(img_directory, image), ori_img)

        with open(label_location, 'w') as fWrite:
            pass
        
        # update label file
        for box in all_boxes:
            with open(label_location, 'a') as fWrite:
                fWrite.writelines(box[0] + ' ' + str(box[1]) + ' ' + str(box[2]) + ' ' + str(box[3]) + ' ' + str(box[4]))
                if all_boxes.index(box) != (len(all_boxes) - 1):
                    fWrite.writelines('\n')

In [ ]:
if small_obj_data_augmentation:
    print('This might take a long time depending on the size of the dataset. Please wait...')
    if not classes_for_small_obj_data_augmentation:
        for aClass in classes:
            smallObjectAugmentation(aClass)
    else:
        for aClass in classes_for_small_obj_data_augmentation:
            smallObjectAugmentation(aClass)

## 2.5 Creating labels in YoLo-FastestV2 format

In [ ]:
# convert OIDv4 labels to YOLO format. From (Class, x1, y1, x2, y2) to (class_num, cx, cy, w_box, h_box)
def YoloLabelCreator(imageDir, labelDir):    
    file_names_images = os.listdir(imageDir)
    file_names_labels = os.listdir(labelDir)
    !cd {os.path.join(os.path.abspath(os.path.join(imageDir, os.pardir)))} && mkdir Yolo_format_labels
    dest_path = os.path.join(os.path.abspath(os.path.join(imageDir, os.pardir)), 'Yolo_format_labels')
    
    if ('Camera' in classes) and ('Mobile phone' in classes) and cam_and_phone_combined:
        if classes.index('Camera') > classes.index('Mobile phone'):
            larger_index = classes.index('Camera')
            smaller_index = classes.index('Mobile phone')
        else:
            larger_index = classes.index('Mobile phone')
            smaller_index = classes.index('Camera')

    for file_name_image in tqdm(file_names_images):
        image = PIL.Image.open(os.path.join(imageDir, file_name_image))
        width, height = image.size
        corresponding_txt = os.path.join(labelDir, file_name_image.strip().split('.')[0] + '.txt')
        with open(corresponding_txt, 'r') as fRead:
            data = fRead.readlines()
            num_obj_in_image = len(data)
            # convert each object to new format
            for obj in range(num_obj_in_image):
                object_data = data[obj]
                object_data_split = object_data.strip().split(' ')
                object_class = object_data_split[0]

                center_x_pixel_obj = float(object_data_split[1]) + (float(object_data_split[3]) - float(object_data_split[1]))/2
                center_y_pixel_obj = float(object_data_split[2]) + (float(object_data_split[4]) - float(object_data_split[2]))/2
                cx_yolo = round(center_x_pixel_obj/width, 4)
                cy_yolo = round(center_y_pixel_obj/height, 4)

                width_pixel_obj = float(object_data_split[3]) - float(object_data_split[1])
                height_pixel_obj = float(object_data_split[4]) - float(object_data_split[2])
                width_yolo = round(width_pixel_obj/width, 4)
                height_yolo = round(height_pixel_obj/height, 4)

                with open(os.path.join(dest_path , file_name_image.strip().split('.')[0]) + '.txt', 'a') as fWrite:
                    if ('Camera' in classes) and ('Mobile phone' in classes) and cam_and_phone_combined:
                        if classes.index(object_class.replace('_', ' ')) == larger_index:
                            object_number = str(smaller_index)
                        elif classes.index(object_class.replace('_', ' ')) > larger_index:
                            object_number = str(classes.index(object_class.replace('_', ' ')) - 1)
                        else: 
                            object_number = str(classes.index(object_class.replace('_', ' ')))
                    else:
                        object_number = str(classes.index(object_class.replace('_', ' ')))
                       
                    # write new labels to a file
                    fWrite.writelines(object_number + ' ' + str(cx_yolo) + ' ' + str(cy_yolo) + ' ' + str(width_yolo) + ' ' + str(height_yolo))
                    if (len(data) > 1):
                        fWrite.writelines('\n')

In [ ]:
print('This might take a long time depending on the size of the dataset. Please wait...')
for aClass in classes:
    if os.path.exists(os.path.join(paths['YOLO_LABELS_' + aClass.upper().replace(' ', '_')])):
        shutil.rmtree(os.path.join(paths['YOLO_LABELS_' + aClass.upper().replace(' ', '_')]))
    YoloLabelCreator(os.path.join(paths['ALL_IMGS_' + aClass.upper().replace(' ', '_')]), os.path.join(paths['ALL_UPDATED_LABELS_' + aClass.upper().replace(' ', '_')]))

## 2.6 Perform 80/20 split between training and validation data

In [ ]:
# performs random 80/20 split and moves the image and all its augmented versions to one of the sets
def trainAndValidationSplit(imageDir, labelDir, aClass):
    TRAIN_IMAGES_LOCAL = os.path.join(os.path.abspath(os.path.join(imageDir, os.pardir)), 'train_imgs_local')
    VAL_IMAGES_LOCAL = os.path.join(os.path.abspath(os.path.join(imageDir, os.pardir)), 'val_imgs_local')
    TRAIN_LABELS_LOCAL = os.path.join(os.path.abspath(os.path.join(imageDir, os.pardir)), 'train_labels_local')
    VAL_LABELS_LOCAL = os.path.join(os.path.abspath(os.path.join(imageDir, os.pardir)), 'val_labels_local')
    !cd {os.path.join(os.path.abspath(os.path.join(imageDir, os.pardir)))} && mkdir train_imgs_local && mkdir val_imgs_local && mkdir train_labels_local && mkdir val_labels_local
    
    if add_data_augmentation:
        # list should have n number of elemenets
        n = increase_class_by_factor[classes.index(aClass)]
    else:
        n = 1
    
    file_names_images = os.listdir(imageDir)    
    file_names_images_aug = [file_names_images[i * n:(i + 1) * n] for i in range((len(file_names_images) + n - 1) // n )] 

    file_names_labels = os.listdir(labelDir)
    file_names_labels_aug = [file_names_labels[i * n:(i + 1) * n] for i in range((len(file_names_labels) + n - 1) // n )] 

    for image_set_index in tqdm(range(len(file_names_images_aug))):
        rand_int = random.randint(0, 9)
        for i in range(n):
            # if chosen as validation image
            if rand_int <= 1:   
                Path(os.path.join(imageDir, file_names_images_aug[image_set_index][i])).rename(os.path.join(VAL_IMAGES_LOCAL, file_names_images_aug[image_set_index][i]))
                Path(os.path.join(labelDir, file_names_labels_aug[image_set_index][i])).rename(os.path.join(VAL_LABELS_LOCAL, file_names_labels_aug[image_set_index][i]))
            # if chosen as train image
            else:
                Path(os.path.join(imageDir, file_names_images_aug[image_set_index][i])).rename(os.path.join(TRAIN_IMAGES_LOCAL, file_names_images_aug[image_set_index][i]))
                Path(os.path.join(labelDir, file_names_labels_aug[image_set_index][i])).rename(os.path.join(TRAIN_LABELS_LOCAL, file_names_labels_aug[image_set_index][i]))

In [ ]:
print('This might take a long time depending on the size of the dataset. Please wait...')
for aClass in classes:
    trainAndValidationSplit(os.path.join(paths['ALL_IMGS_' + aClass.upper().replace(' ', '_')]), os.path.join(paths['YOLO_LABELS_' + aClass.upper().replace(' ', '_')]), aClass)

# 3 Move local files to destination folders

In [ ]:
# if true add images of buildings, car, people(!), etc. without labels. should be only used on datasets only with cameras and phones
# note: if training fails with IndexError: too many indices for tensor of dimension 1, then set it to False
add_filler_images = True

if ('Person' or 'Human face') in classes and add_filler_images:
    raise NameError('Please set add_filler_images to false in this block')

In [ ]:
%%capture
if os.path.exists(os.path.join(paths['TRAIN_IMAGES_DEST'])):
    shutil.rmtree(os.path.join(paths['TRAIN_IMAGES_DEST']))
if os.path.exists(os.path.join(paths['VAL_IMAGES_DEST'])):
    shutil.rmtree(os.path.join(paths['VAL_IMAGES_DEST']))

!cd {paths['YOLO_MAIN_DIR']} && mkdir train && mkdir val

if add_filler_images:
    !xcopy /Y {'\"' + os.path.join('custom_dataset', 'Filler_images', 'train') + '\"'} {paths['TRAIN_IMAGES_DEST']}
    !xcopy /Y {'\"' + os.path.join('custom_dataset', 'Filler_images', 'val') + '\"'} {paths['VAL_IMAGES_DEST']}
# move files
for aClass in classes:
    !xcopy /Y {'\"' + os.path.join(paths['LOCAL_TRAIN_IMGS_' + aClass.upper().replace(' ', '_')]) + '\"'} {paths['TRAIN_IMAGES_DEST']}
    !xcopy /Y {'\"' + os.path.join(paths['LOCAL_VAL_IMGS_' + aClass.upper().replace(' ', '_')]) + '\"'} {paths['VAL_IMAGES_DEST']}
    !xcopy /Y {'\"' + os.path.join(paths['LOCAL_TRAIN_LABELS_' + aClass.upper().replace(' ', '_')]) + '\"'} {paths['TRAIN_IMAGES_DEST']}
    !xcopy /Y {'\"' + os.path.join(paths['LOCAL_VAL_LABELS_' + aClass.upper().replace(' ', '_')]) + '\"'} {paths['VAL_IMAGES_DEST']}

# 4 Generate dataset path .txt files for training and validation

In [ ]:
paths['TRAIN_LIST_DEST'] = os.path.join(paths['YOLO_MAIN_DIR'], 'train.txt')
paths['VAL_LIST_DEST'] = os.path.join(paths['YOLO_MAIN_DIR'], 'val.txt')

if os.path.exists(os.path.join(paths['TRAIN_LIST_DEST'])):
    os.remove(os.path.join(paths['TRAIN_LIST_DEST']))
if os.path.exists(os.path.join(paths['VAL_LIST_DEST'])):
    os.remove(os.path.join(paths['VAL_LIST_DEST']))

# create empty .txt files 
with open(paths['TRAIN_LIST_DEST'], 'w'): pass
with open(paths['VAL_LIST_DEST'], 'w'): pass

In [ ]:
# for each image in train and validation sets, create an entry in .txt file that has the path to the image
file_names_train = os.listdir(paths['TRAIN_IMAGES_DEST'])
for file in file_names_train:
    stripped = file.strip().split('.')
    if(stripped[1] == 'jpg'):
        with open(paths['TRAIN_LIST_DEST'], 'a') as fWrite:
            fWrite.writelines(os.path.join(os.getcwd(), 'Yolo-FastestV2-main', 'train', file))
            if not file == file_names_train[len(file_names_train)-1] and not file == file_names_train[len(file_names_train)-2]:
                fWrite.writelines('\n')
                
file_names_val = os.listdir(paths['VAL_IMAGES_DEST'])
for file in file_names_val:
    stripped = file.strip().split('.')
    if(stripped[1] == 'jpg'):
        with open(paths['VAL_LIST_DEST'], 'a') as fWrite:
            fWrite.writelines(os.path.join(os.getcwd(), 'Yolo-FastestV2-main', 'val', file))
            if not file == file_names_val[len(file_names_val)-1] and not file == file_names_val[len(file_names_val)-2]:
                fWrite.writelines('\n') 

# 5 Generate category.names file

In [ ]:
# create a file with classes names. mobile phone is set as camera if both classes are present 
if os.path.exists(os.path.join(paths['YOLO_DATA_DIR'], 'category.names')):
    os.remove(os.path.join(paths['YOLO_DATA_DIR'], 'category.names'))

if ('Camera' in classes) and ('Mobile phone' in classes) and cam_and_phone_combined:
    if classes.index('Camera') > classes.index('Mobile phone'):
        larger_index = classes.index('Camera')
        smaller_index = classes.index('Mobile phone')
    else:
        larger_index = classes.index('Mobile phone')
        smaller_index = classes.index('Camera')    
    
with open(os.path.join(paths['YOLO_DATA_DIR'], 'category.names'), 'w') as fWrite:
    for classNum in range(len(classes)):
        if ('Camera' in classes) and ('Mobile phone' in classes) and cam_and_phone_combined:
            if classes.index(classes[classNum]) == smaller_index:
                fWrite.writelines('Camera')
            elif classes.index(classes[classNum]) == larger_index:
                continue
            else:   
                fWrite.writelines(classes[classNum])
        else:
            fWrite.writelines(classes[classNum])

        if classNum != (len(classes) - 1):
            fWrite.writelines('\n')

# 6 Get anchor bias

In [ ]:
print('This might take a long time depending on the size of the dataset. Please wait...')
# anchors6.txt file will be generated. make sure your dataset is large, else "nan" is thrown and loops indefinitely
!cd Yolo-FastestV2-main && python genanchors.py --traintxt ./train.txt

# 7 Model training

## 7.1 Set train configuration settings

In [ ]:
# All parameters should be in string format
num_epochs = '300'
learing_rate_base = '0.001'
# Epochs at which base learning rate will be degreased by a factor of 5. E.g., steps = '150,250'
steps = '150, 250'       
batch_size = '128'

## 7.2 Build the training .data configuration file

In [ ]:
# creating configuration file that will be used by train.py script
toWrite = []

if os.path.exists(os.path.join(paths['YOLO_DATA_DIR'], model_name + '.data')):
    os.remove(os.path.join(paths['YOLO_DATA_DIR'], model_name + '.data'))
with open(os.path.join(paths['YOLO_DATA_DIR'], model_name + '.data'), 'w') as fWrite:
    toWrite.append('[name]\n')
    toWrite.append('model_name=' + model_name + '\n')
    toWrite.append('\n')
    toWrite.append('[train-configure]\n')
    toWrite.append('epochs=' + num_epochs + '\n')
    toWrite.append('steps=' + steps + '\n')
    toWrite.append('batch_size=' + batch_size + '\n')
    toWrite.append('subdivisions=1\n')
    toWrite.append('learning_rate=' + learing_rate_base + '\n')
    toWrite.append('\n')
    toWrite.append('[model-configure]\n')
    toWrite.append('pre_weights=None\n')
    if ('Camera' in classes) and ('Mobile phone' in classes) and cam_and_phone_combined:
        toWrite.append('classes=' + str(len(classes) - 1) + '\n')
    else: 
        toWrite.append('classes=' + str(len(classes)) + '\n')
    toWrite.append('width=352\n')
    toWrite.append('height=352\n')
    toWrite.append('anchor_num=3\n')
    with open(os.path.join(paths['YOLO_MAIN_DIR'], 'anchors6.txt'), 'r') as fRead:
        anchors = fRead.readlines()
        toWrite.append('anchors=' + anchors[0])
    toWrite.append('\n')
    toWrite.append('[data-configure]\n')
    toWrite.append('train=./train.txt\n')
    toWrite.append('val=./val.txt\n')
    toWrite.append('names=./data/category.names\n')

    fWrite.writelines(toWrite)

In [ ]:
if os.path.exists(os.path.join(paths['YOLO_MAIN_DIR'], 'loss.txt')):
    os.remove(os.path.join(paths['YOLO_MAIN_DIR'], 'loss.txt'))

## 7.3 Generate training command

In [ ]:
command = ('cd Yolo-FastestV2-main && python {} --data {} && cd ..').format('train.py', os.path.join('data', model_name + '.data'))

In [ ]:
print('Copy the command below and run it in the terminal. Make sure to be in the directory where this notebook is.\n')
print(command)

## 7.4 Plot total loss

In [ ]:
from matplotlib.ticker import FormatStrFormatter

loss = []

with open(os.path.join(paths['YOLO_MAIN_DIR'], "loss.txt"), 'r') as fRead:
    lines = fRead.readlines()
    for line in lines:
        loss.append(float(line[line.find('[')+1:line.find(']')]))

plt.plot(range(1, len(loss)+1), loss)
plt.title("Total loss for class \"Camera\"")
plt.xlabel("Epoch")
plt.ylabel("Total loss")
plt.xticks(np.arange(1, len(loss)+1, 20.0))
plt.yscale('log')
ax = plt.gca()
plt.tick_params(axis='y', which='minor')
ax.yaxis.set_minor_formatter(FormatStrFormatter("%.1f"))
# save figure to main directory
plt.savefig(os.path.join(paths['YOLO_MAIN_DIR'], 'loss.png'))
plt.show()

## 7.5 Copy the final weight file to modelzoo directory

In [ ]:
all_weights_path = os.path.join(paths['YOLO_MAIN_DIR'], 'weights')
files = os.listdir(all_weights_path)
weight_paths = [os.path.join(all_weights_path, basename) for basename in files]
final_weight_file = os.path.basename(max(weight_paths, key=os.path.getctime))
!copy {os.path.join(paths['YOLO_MAIN_DIR'], 'weights', final_weight_file)} {os.path.join(paths['YOLO_MAIN_DIR'], 'modelzoo', final_weight_file)}

In [ ]:
# use if you would like to choose a weight file yourself and moved it yourself to the modelzoo directory. enter the name of that weight file
final_weight_file = 'Yolo_FastestV2_V3_custom_phone_only-290-epoch-0.814434ap-model.pth'     

# 8 Model evaluation

In [ ]:
# evaluates the model based on the wight file belonging to a certain epoch. prints Precision, Recall, AP, and F1 score

In [ ]:
command = ('cd Yolo-FastestV2-main && python {} --data {} --weights {}').format('evaluation.py', os.path.join('data', model_name + '.data'), os.path.join('modelzoo', final_weight_file))

In [ ]:
print('Copy the command below and run it in the terminal. Make sure to be in the directory where this notebook is.\n')
print(command)

# 9 Object detection

## 9.1 Setup

In [ ]:
# modified from https://github.com/dog-qiuqiu/Yolo-FastestV2

In [ ]:
def passImageThroughClassifier(ori_img, cfg):
    # Data preprocessing
    res_img = cv2.resize(ori_img, (cfg["width"], cfg["height"]), interpolation = cv2.INTER_LINEAR) 
    img = res_img.reshape(1, cfg["height"], cfg["width"], 3)
    img = torch.from_numpy(img.transpose(0,3, 1, 2))
    img = img.to(device).float() / 255.0
    
    preds = model(img)

    # Feature map post-processing
    output = utils.utils.handel_preds(preds, cfg, device)
    output_boxes = utils.utils.non_max_suppression(output, conf_thres = 0.3, iou_thres = 0.4)

    h, w, _ = ori_img.shape
    scale_h, scale_w = h / cfg['height'], w / cfg['width']
    
    return output_boxes, scale_h, scale_w

In [ ]:
%%capture
DATA_PATH = os.path.join(paths['YOLO_MAIN_DIR'], 'data', model_name + '.data')
WEIGHTS_PATH = os.path.join(paths['YOLO_MAIN_DIR'], 'modelzoo', final_weight_file)
opt = [DATA_PATH, WEIGHTS_PATH]
cfg = utils.utils.load_datafile(opt[0])
assert os.path.exists(opt[1]), 'Please specify the correct model path'
# model loading
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = det.Detector(cfg['classes'], cfg['anchor_num'], True).to(device)
model.load_state_dict(torch.load(opt[1], map_location=device))
#sets the module in eval node
model.eval()

In [ ]:
LABEL_NAMES = []
with open(os.path.join(paths['YOLO_MAIN_DIR'], 'data', 'category.names'), 'r') as f:
    for line in f.readlines():
         LABEL_NAMES.append(line.strip())

## 9.2 Object detection from images

In [ ]:
# modified from https://github.com/dog-qiuqiu/Yolo-FastestV2
if os.path.exists(os.path.join(paths['YOLO_MAIN_DIR'], 'results', 'output_img')):
    shutil.rmtree(os.path.join(paths['YOLO_MAIN_DIR'], 'results', 'output_img'))

!mkdir {os.path.join(paths['YOLO_MAIN_DIR'], 'results', 'output_img')}

images = os.listdir(os.path.join(paths['YOLO_MAIN_DIR'], 'results', 'input_img'))
for image in images:
    # Data preprocessing
    ori_img = cv2.imread(os.path.join(paths['YOLO_MAIN_DIR'], 'results', 'input_img', image))
    res_img = cv2.resize(ori_img, (cfg['width'], cfg['height']), interpolation = cv2.INTER_LINEAR) 
    img = res_img.reshape(1, cfg['height'], cfg['width'], 3)
    img = torch.from_numpy(img.transpose(0,3, 1, 2))
    img = img.to(device).float() / 255.0

    preds = model(img)

    # Feature map post-processing
    output = utils.utils.handel_preds(preds, cfg, device)
    output_boxes = utils.utils.non_max_suppression(output, conf_thres = 0.3, iou_thres = 0.4)
      
    h, w, _ = ori_img.shape
    scale_h, scale_w = h / cfg['height'], w / cfg['width']

    # Draw the prediction box
    for box in output_boxes[0]:
        box = box.tolist()

        obj_score = box[4]
        if obj_score > 0.6:
            category = LABEL_NAMES[int(box[5])]

            x1, y1 = int(box[0] * scale_w), int(box[1] * scale_h)
            x2, y2 = int(box[2] * scale_w), int(box[3] * scale_h)

            cv2.rectangle(ori_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(ori_img, '%.2f' % obj_score, (x1, y1 - 5), 0, 0.7, (0, 255, 0), 2)	
            cv2.putText(ori_img, category, (x1, y1 - 25), 0, 0.7, (0, 255, 0), 2)

    cv2.imwrite(os.path.join(paths['YOLO_MAIN_DIR'], 'results', 'output_img', image), ori_img)

## 9.3 Real-time object detection

In [ ]:
# modified from https://github.com/dog-qiuqiu/Yolo-FastestV2
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened(): 
    ret, ori_img = cap.read()
    output_boxes, scale_h, scale_w = passImageThroughClassifier(ori_img, cfg)
    
    # Draw the prediction box
    for box in output_boxes[0]:
        box = box.tolist()

        obj_score = box[4]
        if obj_score > 0.5:
            category = LABEL_NAMES[int(box[5])]

            x1, y1 = int(box[0] * scale_w), int(box[1] * scale_h)
            x2, y2 = int(box[2] * scale_w), int(box[3] * scale_h)
            
            cv2.rectangle(ori_img, (x1, y1), (x2, y2), (255, 255, 0), 2)
            cv2.putText(ori_img, '%.2f' % obj_score, (x1, y1 - 5), 0, 0.7, (255, 255, 0), 2)	
            cv2.putText(ori_img, category, (x1, y1 - 25), 0, 0.7, (255, 255, 0), 2)
            cv2.putText(ori_img, category, (x1, y1 - 25), 0, 0.7, (255, 255, 0), 2)

    cv2.imshow('object detection',  ori_img)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

# 10 Deployment

In [ ]:
'''
// Tencent is pleased to support the open source community by making ncnn available.  
//  
// Copyright (C) 2017 THL A29 Limited, a Tencent company. All rights reserved.  
//  
// Licensed under the BSD 3-Clause License (the "License"); you may not use this file except  
// in compliance with the License. You may obtain a copy of the License at  
//  
// https://opensource.org/licenses/BSD-3-Clause  
//  
// Unless required by applicable law or agreed to in writing, software distributed  
// under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR  
// CONDITIONS OF ANY KIND, either express or implied. See the License for the  
// specific language governing permissions and limitations under the License.
'''

## 10.1 Convert pytorch to onnx

In [ ]:
command = ('cd Yolo-FastestV2-main && python {} --data {} --weights {} --output {}').format('pytorch2onnx.py', os.path.join('data', model_name + '.data'), os.path.join('modelzoo', final_weight_file), 'yolo-fastestv2.onnx')

In [ ]:
!{command}

## 10.2 onnx-simplifier

In [ ]:
command = ('cd Yolo-FastestV2-main && python -m onnxsim yolo-fastestv2.onnx yolo-fastestv2-opt.onnx')

In [ ]:
!{command}

## 10.3 Build NCNN

In [ ]:
# this section should only be run once, no need to build NCNN again for every model

In [ ]:
!cd Yolo-FastestV2-main && git clone https://github.com/Tencent/ncnn.git

In [ ]:
'''
This installation is only valid for Windows.  
  
At this step, Jupyter Notebook cannot be used anymore. You have to install Visual Studio 2017 with .NET framework, desktop development with c++ and Windows 10 SDK.   
Download protobuf-3.4.0 from https://github.com/google/protobuf/archive/v3.4.0.zip and create a path variable with the location of the protobuf.  
Download and install Vulkan SDK from https://vulkan.lunarg.com/sdk/home and make sure it is in the path. Go to start and search for x64 Native Tools Command Prompt for VS 2017 and run it as administrator (!).  
Then build protobuf library:  
  
cd <protobuf-root-dir>  
mkdir build  
cd build  
cmake -G"NMake Makefiles" -DCMAKE_BUILD_TYPE=Release -DCMAKE_INSTALL_PREFIX=%cd%/install -Dprotobuf_BUILD_TESTS=OFF -Dprotobuf_MSVC_STATIC_RUNTIME=OFF ../cmake  
nmake  
nmake install  
      
And build ncnn by going to the directory of this notebook and using:  
      
cd Yolo-FastestV2-main  
cd ncnn  
mkdir build  
cd build  
cmake .. -G"NMake Makefiles"  
nmake  
nmake install  
  
If everything went well, you should see 'install' folder in ncnn/build directory.
'''

In [ ]:
# copy bin, include and lib from install folder to sample/ncnn folder
if os.path.exists(os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'install')):
    shutil.copytree(os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'install', 'bin'), os.path.join(paths['YOLO_MAIN_DIR'], 'sample', 'ncnn', 'bin'))
    shutil.copytree(os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'install', 'include'), os.path.join(paths['YOLO_MAIN_DIR'], 'sample', 'ncnn', 'include'))
    shutil.copytree(os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'install', 'lib'), os.path.join(paths['YOLO_MAIN_DIR'], 'sample', 'ncnn', 'lib'))

## 10.4 Conversion to NCNN

In [ ]:
# copy yolo-fastestv2-opt.onnx in main directory to ncnn/build/tools/onnx
if os.path.exists(os.path.join(paths['YOLO_MAIN_DIR'], 'yolo-fastestv2-opt.onnx')):
    !copy {os.path.join(paths['YOLO_MAIN_DIR'], 'yolo-fastestv2-opt.onnx')} {os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'tools', 'onnx', 'yolo-fastestv2-opt.onnx')}

In [ ]:
# execute onnx2ncnn file in ncnn/build/tools/onnx and generate yolo-fastestv2.bin and yolo-fastestv2.param
!cd Yolo-FastestV2-main/ncnn/build/tools/onnx && onnx2ncnn yolo-fastestv2-opt.onnx yolo-fastestv2.param yolo-fastestv2.bin 

In [ ]:
# copy all yolo-fastestv2* files to parent directory (tools)
!copy {os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'tools', 'onnx', 'yolo-fastestv2-opt.onnx')} {os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'tools', 'yolo-fastestv2-opt.onnx')}
!copy {os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'tools', 'onnx', 'yolo-fastestv2.bin')} {os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'tools', 'yolo-fastestv2.bin')}
!copy {os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'tools', 'onnx', 'yolo-fastestv2.param')} {os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'tools', 'yolo-fastestv2.param')}
os.remove(os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'tools', 'onnx', 'yolo-fastestv2-opt.onnx'))
os.remove(os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'tools', 'onnx', 'yolo-fastestv2.bin'))
os.remove(os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'tools', 'onnx', 'yolo-fastestv2.param'))

In [ ]:
# execute ncnnoptimize on yolo-fastestv2.param and yolo-fastestv2.bin
!cd Yolo-FastestV2-main/ncnn/build/tools/ && ncnnoptimize yolo-fastestv2.param yolo-fastestv2.bin yolo-fastestv2-opt.param yolo-fastestv2-opt.bin 1

In [ ]:
# copy generated optimized yolo-fastestv2-opt* files to sample/ncnn/model folder
!copy {os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'tools', 'yolo-fastestv2-opt.bin')} {os.path.join(paths['YOLO_MAIN_DIR'], 'sample', 'ncnn', 'model', 'yolo-fastestv2-opt.bin')}
!copy {os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'tools', 'yolo-fastestv2-opt.param')} {os.path.join(paths['YOLO_MAIN_DIR'], 'sample', 'ncnn', 'model', 'yolo-fastestv2-opt.param')}

In [ ]:
# copy onnx file too
!copy {os.path.join(paths['YOLO_MAIN_DIR'], 'yolo-fastestv2-opt.onnx')} {os.path.join(paths['YOLO_MAIN_DIR'], 'sample', 'ncnn', 'model', 'yolo-fastestv2-opt.onnx')}

## 10.5 Quantize NCNN

In [ ]:
# copy generated optimized yolo-fastestv2-opt* files to quantize folder folder
!copy {os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'tools', 'yolo-fastestv2-opt.bin')} {os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'tools', 'quantize', 'yolo-fastestv2-opt.bin')}
!copy {os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'tools', 'yolo-fastestv2-opt.param')} {os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'tools', 'quantize', 'yolo-fastestv2-opt.param')}
os.remove(os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'tools', 'yolo-fastestv2-opt.onnx'))
os.remove(os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'tools', 'yolo-fastestv2.bin'))
os.remove(os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'tools', 'yolo-fastestv2.param'))
os.remove(os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'tools', 'yolo-fastestv2-opt.bin'))
os.remove(os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'tools', 'yolo-fastestv2-opt.param'))

In [ ]:
# download 1000 ImageNet images  for calibration of table file
!cd {os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'tools', 'quantize')} && git clone https://github.com/EliSchwartz/imagenet-sample-images

In [ ]:
file_names = os.listdir(os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'tools', 'quantize', 'imagenet-sample-images'))
# remove README file
if file_names[len(file_names) - 1] == 'README.md':
    os.remove(os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'tools', 'quantize', 'imagenet-sample-images', file_names[len(file_names) - 1]))

In [ ]:
# create imagelist.txt file
with open(os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'tools', 'quantize', 'imagelist.txt'), 'w') as fWrite:
    for file in file_names:
        if file == '.git':
            continue
        fWrite.writelines('imagenet-sample-images/')
        fWrite.writelines(file)
        if file != file_names[len(file_names) - 1]:
            fWrite.writelines('\n')

In [ ]:
# create the calibration table file
!cd Yolo-FastestV2-main/ncnn/build/tools/quantize/ && ncnn2table yolo-fastestv2-opt.param yolo-fastestv2-opt.bin imagelist.txt yolo-fastestv2-opt.table mean=[0,0,0] norm=[0.0039,0.0039,0.0039] shape=[352,352,3] pixel=RGB thread=8 method=kl

In [ ]:
# quantize model
!cd Yolo-FastestV2-main/ncnn/build/tools/quantize/ && ncnn2int8 yolo-fastestv2-opt.param yolo-fastestv2-opt.bin yolo-fastestv2-int8.param yolo-fastestv2-int8.bin yolo-fastestv2-opt.table

In [ ]:
# copy generated yolo-fastestv2-int8* files to sample/ncnn/model folder
!copy {os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'tools', 'quantize', 'yolo-fastestv2-int8.bin')} {os.path.join(paths['YOLO_MAIN_DIR'], 'sample', 'ncnn', 'model', 'yolo-fastestv2-int8.bin')}
!copy {os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'tools', 'quantize', 'yolo-fastestv2-int8.param')} {os.path.join(paths['YOLO_MAIN_DIR'], 'sample', 'ncnn', 'model', 'yolo-fastestv2-int8.param')}
os.remove(os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'tools', 'quantize', 'yolo-fastestv2-int8.bin'))
os.remove(os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'tools', 'quantize', 'yolo-fastestv2-int8.param'))
os.remove(os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'tools', 'quantize', 'yolo-fastestv2-opt.bin'))
os.remove(os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'tools', 'quantize', 'yolo-fastestv2-opt.param'))
os.remove(os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'tools', 'quantize', 'yolo-fastestv2-opt.table'))

In [ ]:
# delete the calibration folder with images
#if os.path.exists(os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'tools', 'quantize', 'imagenet-sample-images')):
#    os.system('rmdir /S /Q "{}"'.format(os.path.join(paths['YOLO_MAIN_DIR'], 'ncnn', 'build', 'tools', 'quantize', 'imagenet-sample-images')))

In [ ]:
# at this step files in sample/ncnn/model folder can be copied and sent to Pi to be used. MNN is made on Pi, so onnx file will be needed for it. Pi has instructions on how to make MNN model from onnx

## 10.6 Test NCNN on Windows

In [ ]:
# disclaimer: this section and next one are made for NCNN to run on windows. the image of the Pi already contains NCNN that can be run on Linux

In [ ]:
'''
Instructions below are there to test custom models from scratch on NCNN on Windows. Yolo-FastestV2-main\sample\ directory already contains codes for detection of humans and cameras (see YoLo-FastestV2-cpp-TEST and YoLo-FastestV2-cpp-REAL-TIME).
If you would like to test other classes, demo files need to be altered accordingly. 
'''

In [ ]:
'''
At this stage more efficient C++ implementation will be tested. Download OpenCV here https://opencv.org/releases/ and install it in a directory of your choice. 
Next, create a Visual Studio project (2017 was used) named YoLo-FastestV2-cpp-TEST. Place this project inside sample directory ("Yolo-FastestV2-main\sample\YoLo-FastestV2-cpp-TEST"). 
Create a blank project. Run the next cell.
'''

In [ ]:
# copy latest model files to NCNN image object detection folder
if not os.path.exists(os.path.join(paths['YOLO_MAIN_DIR'], 'sample', 'ncnn', 'YoLo-FastestV2-cpp-TEST' , 'model')):
    shutil.copytree(os.path.join(paths['YOLO_MAIN_DIR'], 'sample', 'ncnn', 'model'), os.path.join(paths['YOLO_MAIN_DIR'], 'sample', 'ncnn', 'YoLo-FastestV2-cpp-TEST' , 'model'))
if not os.path.exists(os.path.join(paths['YOLO_MAIN_DIR'], 'sample', 'ncnn', 'YoLo-FastestV2-cpp-TEST' , 'src')):
    shutil.copytree(os.path.join(paths['YOLO_MAIN_DIR'], 'sample', 'ncnn', 'src'), os.path.join(paths['YOLO_MAIN_DIR'], 'sample', 'ncnn', 'YoLo-FastestV2-cpp-TEST' , 'src'))
if not os.path.exists(os.path.join(paths['YOLO_MAIN_DIR'], 'sample', 'ncnn', 'YoLo-FastestV2-cpp-TEST' , 'demo.cpp')):
    !copy {os.path.join(paths['YOLO_MAIN_DIR'], 'sample', 'ncnn', 'demo.cpp')} {os.path.join(paths['YOLO_MAIN_DIR'], 'sample', 'ncnn', 'YoLo-FastestV2-cpp-TEST' , 'demo.cpp')}
if not os.path.exists(os.path.join(paths['YOLO_MAIN_DIR'], 'sample', 'ncnn', 'YoLo-FastestV2-cpp-TEST' , 'test.jpg')):
    !copy {os.path.join(paths['YOLO_MAIN_DIR'], 'sample', 'ncnn', 'test.jpg')} {os.path.join(paths['YOLO_MAIN_DIR'], 'sample', 'ncnn', 'YoLo-FastestV2-cpp-TEST' , 'test.jpg')}

In [ ]:
'''
Once a blank project was created, set it to Release x64 (left of run button) and go to View -> Property Manager. 
Select Release | x64 and a window will open. Go to Common Properties -> VC++ Directories -> Include Directories and add:

<your project path>/Yolo-FastestV2-main/ncnn/build/install/include/ncnn;
<your openCV path>/opencv/build/include;
<your openCV path>/opencv/build/include/opencv;
<your openCV path>/opencv/build/include/opencv2;
<your protobuf path>/protobuf-3.4.0/build/install/include;
<your project path>/YoLo-FastestV2-cpp-TEST/src/include;
<your project path>/YoLo-FastestV2-cpp-TEST/include/ncnn;

Make sure that you already built protobuf library in previuos steps. 
Next, Go to Common Properties -> VC++ Directories -> Library Directories and add:

<your openCV path>/opencv/build/x64/vc15/lib;
<your project path>/Yolo-FastestV2-main/ncnn/build/install/lib;
<your protobuf path>/protobuf-3.4.0/build/lib;

Then go to Common Properties -> Linker -> Input -> Additional Dependencies and add:

ncnn.lib;
libprotobuf.lib;
opencv_world455.lib;

Once done your folder architecture should look like this:

<your project path>/Yolo-FastestV2-main/sample/ncnn/YoLo-FastestV2-cpp-TEST

├── model 
│   ├── model bin and param files
├── src
│   ├── yolo-fastestv2.cpp
│   ├── yolo-fastestv2.h
├── demo.cpp
├── test.jpg
├── YoLo-FastestV2-cpp-TEST.vcxproj
├── ... (rest of Visual Studio files)

Run the project. output.png will be generated in ...\sample\ncnn\YoLo-FastestV2-cpp-TEST directory.
'''

## 10.7 Real-time object detection with NCNN

In [ ]:
'''
Create a Visual Studio project (2017 was used) named YoLo-FastestV2-cpp-REAL-TIME. 
Place this project inside sample directory ("Yolo-FastestV2-main\sample\YoLo-FastestV2-cpp-REAL-TIME"). Create a blank project. 
Run the next cell.
'''

In [ ]:
# copy latest model files to NCNN real-time object detection folder
if not os.path.exists(os.path.join(paths['YOLO_MAIN_DIR'], 'sample', 'ncnn', 'YoLo-FastestV2-cpp-REAL-TIME' , 'model')):
    shutil.copytree(os.path.join(paths['YOLO_MAIN_DIR'], 'sample', 'ncnn', 'model'), os.path.join(paths['YOLO_MAIN_DIR'], 'sample', 'ncnn', 'YoLo-FastestV2-cpp-REAL-TIME' , 'model'))
if not os.path.exists(os.path.join(paths['YOLO_MAIN_DIR'], 'sample', 'ncnn', 'YoLo-FastestV2-cpp-REAL-TIME' , 'src')):
    shutil.copytree(os.path.join(paths['YOLO_MAIN_DIR'], 'sample', 'ncnn', 'src'), os.path.join(paths['YOLO_MAIN_DIR'], 'sample', 'ncnn', 'YoLo-FastestV2-cpp-REAL-TIME' , 'src'))
if not os.path.exists(os.path.join(paths['YOLO_MAIN_DIR'], 'sample', 'ncnn', 'YoLo-FastestV2-cpp-REAL-TIME' , 'real-time_demo.cpp')):
    !copy {os.path.join(paths['YOLO_MAIN_DIR'], 'sample', 'ncnn', 'real-time_demo.cpp')} {os.path.join(paths['YOLO_MAIN_DIR'], 'sample', 'ncnn', 'YoLo-FastestV2-cpp-REAL-TIME' , 'real-time_demo.cpp')}

In [ ]:
'''
The procedure is the same as for Test NCNN section. The only difference is now demo.cpp is replaced by real-time_demo.cpp. 
By running this scrip you will have a window showing the webcam view of the device and if camera is detected in the frame, a rectangle will be drawn around it.
'''

# 11 Test set generation for PI

In [ ]:
# same order as in section 1.2
limit_test_images_to = [10, 10]

In [ ]:
print('Copy command(s) below and run in the terminal. Make sure to be in the directory where this notebook is.\n')

for classNum in range(len(classes)):
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    command = ('python OIDv4_ToolKit/main.py downloader --classes ' + '\"' + classes[classNum] + '\"' + ' --type_csv test --limit ' + str(limit_test_images_to[classNum]))
    print(command)
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

## 11.1 Creating label files with multiple objects in an image

In [ ]:
# run class_code block in section 2.2 if you didn't yet

In [ ]:
df_test = pd.read_csv(os.path.join('OID', 'csv_folder', 'test-annotations-bbox.csv'))
df_classes = pd.read_csv(os.path.join('OID', 'csv_folder', 'class-descriptions-boxable.csv'), header=None)

In [ ]:
groups_dict_test = {}
keys_test = ['0'] * len(classes)
for i in range(len(classes)):
    keys_test[i] = 'groups_' + classes[i]

values_test = class_code
for i in range(len(keys_test)):
    groups_dict_test[keys_test[i]] = df_test[(df_test.LabelName == values_test[i])].groupby(df_test.ImageID)

In [ ]:
for aClass in classes:
    paths['TEST_DIR'] = os.path.join('OID', 'Dataset', 'test')
    paths['TEST_' + aClass.upper().replace(' ', '_')] = os.path.join(paths['TEST_DIR'], aClass)
    paths['ALL_TEST_IMGS_' + aClass.upper().replace(' ', '_')] = os.path.join(paths['TEST_DIR'], aClass, aClass.replace(' ', '_') +'_images')
    paths['ALL_UPDATED_TEST_LABELS_' + aClass.upper().replace(' ', '_')] = os.path.join(paths['TEST_DIR'], aClass, 'Label_updated')

In [ ]:
# put images in one folder for a clean directory structure
for aClass in classes:
    # remove label folder since updated version will be made
    if os.path.exists(os.path.join(paths['TEST_' + aClass.upper().replace(' ', '_')], 'Label')):
        shutil.rmtree(os.path.join(paths['TEST_' + aClass.upper().replace(' ', '_')], 'Label'))
    !cd {os.path.join(paths['TEST_' + aClass.upper().replace(' ', '_')])} && mkdir {aClass.replace(' ', '_')  + '_images'}
    image_list = [f.split('.')[0] for f in os.listdir(os.path.join(paths['TEST_DIR'], aClass)) if f.endswith('.jpg')]
    for image in image_list:
        os.rename(os.path.join(paths['TEST_' + aClass.upper().replace(' ', '_')], image + '.jpg'), os.path.join(paths['ALL_TEST_IMGS_' + aClass.upper().replace(' ', '_')], image + '.jpg'))

In [ ]:
# are mobile phone and camera in one "Camera" class?
combine_phone_and_camera = False

In [ ]:
# slightly different for test
def labelUpdaterTest(): 
    groups = list(groups_dict_test.values())
    for i in range(len(groups)):
        # copies are made to not affect original lists when reshuffling entries in them
        copy_groups = list.copy(groups)
        copy_groups.insert(0, copy_groups.pop(copy_groups.index(copy_groups[i])))
        copy_classes = list.copy(classes)
        copy_classes.insert(0, copy_classes.pop(copy_classes.index(copy_classes[i])))

        downloaded_images_list = [f.split('.')[0] for f in os.listdir(os.path.join(paths['ALL_TEST_IMGS_' + copy_classes[0].upper().replace(' ', '_')])) if f.endswith('.jpg')]
        images_label_list = list(set(downloaded_images_list))

        for image in downloaded_images_list:
            try:
                current_image_path = os.path.join(paths['ALL_TEST_IMGS_' + copy_classes[0].upper().replace(' ', '_')], image + '.jpg')
                dataset_image = cv2.imread(current_image_path)
                boxes= copy_groups[0].get_group(image.split('.')[0])[['XMin', 'XMax', 'YMin', 'YMax']].values.tolist()
                file_name = str(image.split('.')[0]) + '.txt'
                file_path = os.path.join(paths['ALL_UPDATED_TEST_LABELS_' + copy_classes[0].upper().replace(' ', '_')], file_name)
                if os.path.isfile(file_path):
                    f = open(file_path, 'a')
                else:
                    f = open(file_path, 'w')

                for box in boxes:
                    box[0] *= int(dataset_image.shape[1])
                    box[1] *= int(dataset_image.shape[1])
                    box[2] *= int(dataset_image.shape[0])
                    box[3] *= int(dataset_image.shape[0])

                    # each row in a file is name of the class_name, XMin, YMix, XMax, YMax (left top right bottom)
                    if combine_phone_and_camera and copy_classes[0] == "Mobile phone":
                        print("Camera", box[0], box[2], box[1], box[3], file=f)
                    else:
                        print(copy_classes[0].replace(' ', '_'), box[0], box[2], box[1], box[3], file=f)

                for categoryNum in range(len(copy_classes) - 1):
                    try:
                        boxes = copy_groups[categoryNum + 1].get_group(image.split('.')[0])[['XMin', 'XMax', 'YMin', 'YMax']].values.tolist()

                        for box in boxes:
                            box[0] *= int(dataset_image.shape[1])
                            box[1] *= int(dataset_image.shape[1])
                            box[2] *= int(dataset_image.shape[0])
                            box[3] *= int(dataset_image.shape[0])

                        # each row in a file is name of the class_name, XMin, YMix, XMax, YMax (left top right bottom)
                        if combine_phone_and_camera and copy_classes[categoryNum + 1] == "Mobile phone":
                            print("Camera", box[0], box[2], box[1], box[3], file=f)
                        else:
                            print(copy_classes[categoryNum + 1].replace(' ', '_'), box[0], box[2], box[1], box[3], file=f)
                    except Exception as e:
                        pass    

            except Exception as e:
                pass

In [ ]:
# searching labels in a file with hundreds of millions of entries, will take some time
print('This might take a long time depending on the size of the dataset. Please wait...')    
for aClass in classes:
    if os.path.exists(os.path.join(paths['ALL_UPDATED_TEST_LABELS_' + aClass.upper().replace(' ', '_')])):
        shutil.rmtree(os.path.join(paths['ALL_UPDATED_TEST_LABELS_' + aClass.upper().replace(' ', '_')]))
    !cd {os.path.join(paths['TEST_' + aClass.upper().replace(' ', '_')])} && mkdir Label_updated
labelUpdaterTest()